In [28]:
import pandas as pd

from orion.data import load_signal
from orion import Orion
from orion.data import load_anomalies

from mlprimitives.custom.timeseries_preprocessing import time_segments_aggregate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from mlprimitives.custom.timeseries_preprocessing import rolling_window_sequences
from orion.primitives.timeseries_preprocessing import slice_array_by_dims
from mlprimitives import load_primitive
import numpy as np
from orion.primitives.tadgan import TadGAN
import tensorflow as tf
import keras
from keras.models import Model
from keras import Input
from functools import partial

import logging
import math
import tempfile
from functools import partial

import keras
import numpy as np
import pandas as pd
from keras import backend as K
from keras.layers import Input
from keras.layers.merge import _Merge
from keras.models import Model
from mlprimitives.adapters.keras import build_layer
from mlprimitives.utils import import_object
from scipy import stats


# Orion Shortcut

In [ ]:
# this is a reconstruction model, namely Generative Adversarial Networks (GAN)
params = {
    "epochs": 5, 
    "input_shape":[100, 25], 
    "target_shape": [100, 1]
}

primitive = load_primitive('orion.primitives.tadgan.TadGAN', 
                           arguments=params)
primitive.fit(X=X, y=y)
y_hat, critic = primitive.produce(X=X, y=y)

In [16]:
np.save('processed/tadgan/y_hat', y_hat)
np.save('processed/tadgan/critic', critic)

# Loading the dataset

In [82]:
X = np.load('processed/data/X.npy')
target = np.load('processed/data/y.npy')
X.shape, y.shape

((2718, 100, 25), (64, 100, 1))

# Hyperparameter

In [67]:
input_shape=(100, 25)
target_shape=(100, 1)
latent_dim=20
learning_rate=0.0005
epochs=70
batch_size=64
iterations_critic=5
latent_shape = (latent_dim, 1)

shape = np.asarray(X)[0].shape
length = shape[0]
target_shape = np.asarray(target)[0].shape


generator_reshape_dim =  length // 2
generator_reshape_shape = (length // 2, 1)
encoder_reshape_shape = latent_shape

encoder_input_shape = shape
generator_input_shape = latent_shape
critic_x_input_shape = target_shape
critic_z_input_shape = latent_shape



lstm_units = 100
dense_units = 20

# Encoder

In [68]:
def build_encoder(input_shape, lstm_units, dense_units, encoder_reshape_shape):
    x = Input(shape=input_shape)
    model = keras.models.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=lstm_units, return_sequences=True)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=dense_units))
    model.add(keras.layers.Reshape(target_shape=encoder_reshape_shape))
    return Model(x, model(x))
              
encoder = build_encoder(
    input_shape=encoder_input_shape,
    lstm_units=lstm_units,
    dense_units=dense_units,
    encoder_reshape_shape=encoder_reshape_shape,
)
encoder.summary()

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_67 (InputLayer)        (None, 100, 25)           0         
_________________________________________________________________
sequential_18 (Sequential)   (None, 20, 1)             500820    
Total params: 500,820
Trainable params: 500,820
Non-trainable params: 0
_________________________________________________________________


# Generator

In [69]:
def build_generator(input_shape, generator_reshape_dim, generator_reshape_shape):
    x = Input(shape=input_shape)
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=generator_reshape_dim))
    model.add(keras.layers.Reshape(target_shape=generator_reshape_shape))
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(keras.layers.convolutional.UpSampling1D(size=2))
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=1)))
    model.add(keras.layers.Activation(activation='tanh'))
    return Model(x, model(x))
              
generator = build_generator(
    input_shape=generator_input_shape,
    generator_reshape_dim=generator_reshape_dim,
    generator_reshape_shape=generator_reshape_shape,
)
generator.summary()

Model: "model_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_68 (InputLayer)        (None, 20, 1)             0         
_________________________________________________________________
sequential_19 (Sequential)   (None, 100, 1)            133787    
Total params: 133,787
Trainable params: 133,787
Non-trainable params: 0
_________________________________________________________________


# Critic X

In [70]:
def build_critic_x(input_shape):
    x = Input(shape=input_shape)
    model = keras.models.Sequential()
    model.add(keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(rate=0.25))
    model.add(keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(rate=0.25))
    model.add(keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(rate=0.25))
    model.add(keras.layers.Conv1D(filters=64, kernel_size=5))
    model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(rate=0.25))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=1))
    return Model(x, model(x))
              
critic_x = build_critic_x(
    input_shape=critic_x_input_shape
)
critic_x.summary()

Model: "model_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_69 (InputLayer)        (None, 100, 1)            0         
_________________________________________________________________
sequential_20 (Sequential)   (None, 1)                 67393     
Total params: 67,393
Trainable params: 67,393
Non-trainable params: 0
_________________________________________________________________


# Critics Z

In [71]:
def build_critic_z(input_shape, dense_units=20):
    x = Input(shape=input_shape)
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=dense_units))
    model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(units=dense_units))
    model.add(keras.layers.advanced_activations.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(units=1))
    return Model(x, model(x))
              
critic_z = build_critic_z(
    input_shape=critic_z_input_shape,
)
critic_z.summary()

Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_70 (InputLayer)        (None, 20, 1)             0         
_________________________________________________________________
sequential_21 (Sequential)   (None, 1)                 861       
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________


In [72]:
optimizer = keras.optimizers.Adam(learning_rate)

# TadGAN

In [73]:
class RandomWeightedAverage(_Merge):
    def _merge_function(self, inputs):
        """
        Args:
            inputs[0] x     original input
            inputs[1] x_    predicted input
        """
        alpha = K.random_uniform((64, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

def _wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def _gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [74]:
generator.trainable = False
encoder.trainable = False

x = Input(shape=input_shape)
y = Input(shape=target_shape)
z = Input(shape=(latent_dim, 1))

x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x(x_) # Fake
valid_x = critic_x(y) # Truth

## Critic X

In [75]:
interpolated_x = RandomWeightedAverage()([y, x_])
validity_interpolated_x = critic_x(interpolated_x)
partial_gp_loss_x = partial(_gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_model = Model(inputs=[y, z], outputs=[valid_x, fake_x,validity_interpolated_x])
critic_x_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, partial_gp_loss_x], 
                       optimizer=optimizer, loss_weights=[1, 1, 10])

## Critic Z

In [76]:
fake_z = critic_z(z_)
valid_z = critic_z(z)
interpolated_z = RandomWeightedAverage()([z, z_])
validity_interpolated_z = critic_z(interpolated_z)
partial_gp_loss_z = partial(_gradient_penalty_loss, averaged_samples=interpolated_z)
partial_gp_loss_z.__name__ = 'gradient_penalty'
critic_z_model = Model(inputs=[x, z], outputs=[valid_z, fake_z,validity_interpolated_z])
critic_z_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,
                                  partial_gp_loss_z], optimizer=optimizer,
                            loss_weights=[1, 1, 10])

## Encoder-Generator Model

In [77]:
critic_x.trainable = False
critic_z.trainable = False
generator.trainable = True
encoder.trainable = True

z_gen = Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_gen = Input(shape=input_shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x(x_gen_)
fake_gen_z = critic_z(z_gen_)

encoder_generator_model = Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])
encoder_generator_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,'mse'], 
                                optimizer=optimizer,
                                loss_weights=[1, 1, 10])

# Training

In [81]:
fake = np.ones((batch_size, 1))
valid = -np.ones((batch_size, 1))
delta = np.ones((batch_size, 1))

indices = np.arange(X.shape[0])
for epoch in range(1, epochs + 1):
    np.random.shuffle(indices)
    X_ = X[indices]
    y_ = target[indices]

    epoch_g_loss = []
    epoch_cx_loss = []
    epoch_cz_loss = []

    minibatches_size = batch_size * iterations_critic
    num_minibatches = int(X_.shape[0] // minibatches_size)

    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size: (i + 1) * minibatches_size]
        y_minibatch = y_[i * minibatches_size: (i + 1) * minibatches_size]

        for j in range(iterations_critic):
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            y = y_minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            epoch_cx_loss.append(
                critic_x_model.train_on_batch([y, z], [valid, fake, delta]))
            epoch_cz_loss.append(
                critic_z_model.train_on_batch([x, z], [valid, fake, delta]))

        epoch_g_loss.append(
            encoder_generator_model.train_on_batch([x, z], [valid, valid, y]))

    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    print('Epoch: {}/{}, [Dx loss: {}] [Dz loss: {}] [G loss: {}]'.format(
        epoch, epochs, cx_loss, cz_loss, g_loss))

Epoch: 1/70, [Dx loss: [-6.4244094  -8.990245    1.4497118   0.11161248]] [Dz loss: [-1.5030787   0.00510997 -2.1871135   0.06789251]] [G loss: [31.949804  -1.1597579 28.382097   0.4727465]]
Epoch: 2/70, [Dx loss: [-6.2216744  -8.056215    0.7942165   0.10403246]] [Dz loss: [-198.43172       0.8158039  -201.48816       0.22406058]] [G loss: [237.2679     -0.914989  233.7651      0.4417783]]
Epoch: 3/70, [Dx loss: [-5.7019663  -9.756934    3.1240113   0.09309564]] [Dz loss: [-67.72063     1.305833  -79.132935    1.0106499]] [G loss: [97.81973    -3.1382203  96.72089     0.42370585]]
Epoch: 4/70, [Dx loss: [-5.8628798  -9.339897    2.5638502   0.09131683]] [Dz loss: [39.571148    1.526362   36.12168     0.19231087]] [G loss: [-23.700974    -2.4634533  -25.509037     0.42715144]]
Epoch: 5/70, [Dx loss: [-6.2293406  -9.699234    2.476926    0.09929674]] [Dz loss: [ -8.266696     1.6569145  -10.943901     0.10202897]] [G loss: [21.535471   -2.3860874  19.681488    0.42400715]]
Epoch: 6/70, 

# Inference

In [91]:
def predict(X, y=None):
    """Predict values using the initialized object.

    Args:
        X (ndarray):
            N-dimensional array containing the input sequences for the model.
        y (ndarray):
            N-dimensional array containing the target sequences we want to reconstruct.

    Returns:
        ndarray:
            N-dimensional array containing the reconstructions for each input sequence.
        ndarray:
            N-dimensional array containing the critic scores for each input sequence.
    """
    if y is None:
        y = X.copy()  # reconstruct the same input

    z_ = encoder.predict(X)
    y_hat = generator.predict(z_)
    critic = critic_x.predict(y)

    return y_hat, critic

In [93]:
y_hat, critic = predict(X, target)

In [95]:
np.save('processed/tadgan/y_hat', y_hat)
np.save('processed/tadgan/critic', critic)